In [1]:
import pandas as pd
import numpy as np
import sklearn

from xgboost.sklearn import XGBRegressor
from sklearn.datasets import make_classification
import xgboost as xgboost
from sklearn.model_selection import GridSearchCV

In [8]:
train = pd.read_csv('featuresTrain4.csv')
test = pd.read_csv('featuresTest4.csv')

In [9]:
train.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,provincia_24,provincia_25,provincia_26,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,provincia_33
0,254099,Apartamento,Benito Juárez,Distrito Federal,2.210044,2.0,1.0,2.0,4.394449,4.394449,...,0,0,0,0,0,0,0,0,0,0
1,53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,2.397895,3.0,2.0,2.0,5.594711,5.198497,...,0,0,0,0,0,0,0,0,0,0
2,247984,Casa,Tonalá,Jalisco,1.791759,3.0,2.0,2.0,4.976734,5.117994,...,0,0,0,0,0,0,0,0,0,0
3,209067,Casa,Zinacantepec,Edo. de México,0.693147,2.0,1.0,1.0,4.158883,4.219508,...,0,0,0,0,0,0,0,0,0,0
4,185997,Apartamento,Zapopan,Jalisco,2.397895,2.0,1.0,1.0,4.564348,4.564348,...,0,0,0,0,0,0,0,0,0,0


In [10]:
y = train['precio']
train = train.drop(['id','precio','ciudad','provincia','tipodepropiedad'],1)
train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,...,provincia_24,provincia_25,provincia_26,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,provincia_33
0,2.210044,2.0,1.0,2.0,4.394449,4.394449,23533.0,20.696818,-99.509221,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2.397895,3.0,2.0,2.0,5.594711,5.198497,24514.0,19.310205,-99.227655,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1.791759,3.0,2.0,2.0,4.976734,5.117994,48551.0,20.696818,-99.509221,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.693147,2.0,1.0,1.0,4.158883,4.219508,53666.0,19.301890,-99.688015,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2.397895,2.0,1.0,1.0,4.564348,4.564348,47835.0,20.696818,-99.509221,0.0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y.head()

0    2273000.0
1    3600000.0
2    1200000.0
3     650000.0
4    1150000.0
Name: precio, dtype: float64

In [12]:
# primero voy a definir un xgboost con valores promerios para poder tunearlo:
modelo_xgb = xgboost.XGBRegressor(learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=5,
 gamma=0.1,
 subsample=0.95,
 colsample_bytree=0.80,
 objective= 'reg:squarederror',
 nthread=4,
 scale_pos_weight=1)

In [13]:
#Entreno dicho xgboost:
modelo_xgb.fit(train,y)

/home/alfonso/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/alfonso/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.1,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=5, missing=None, n_estimators=1000,
             n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.95, verbosity=1)

In [14]:
# esto nos ayudara a tunear el algoritmo, la idea es que tunearlo con los valores de la lista:
parameters_for_testing = {
    'colsample_bytree':[0.85],
    'gamma':[0.1],
    'min_child_weight':[2],               # 4
    'learning_rate':[0.01],
    'max_depth':[8],                           # 2
    'n_estimators':[1000],#10,10100,1000
#    'reg_alpha':[0.03,0.04,0.05,0.06,0.07],#1e-5, 1e-2
#    'reg_lambda':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
    'subsample':[1]
}

In [ ]:
# defino el gridsearch y lo aplico en el xgboost que defini para tunearlo:

gsearch1 = GridSearchCV(estimator = modelo_xgb, param_grid = parameters_for_testing,\
                        n_jobs=4,cv=3,iid=False, verbose=10,scoring='neg_mean_absolute_error',refit=True)
gsearch1.fit(train,y)
#print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
# con los datos del gridsearch defino el mejor modelo de xgboost que encontre:
mejor_xgb =  xgboost.XGBRegressor(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=7,
     min_child_weight=2,
     gamma=0.1,
     subsample=1,
     colsample_bytree=0.85,
     objective= 'reg:squarederror',
     nthread=4,
     scale_pos_weight=1)

In [ ]:
#Entreno el modelo:
mejor_xgb.fit(train,y)

In [ ]:
ids = test['id']
test = test.drop(['id','provincia','ciudad'],axis = 1)

In [ ]:
# realizo la prediccion:
prediction = mejor_xgb.predict(test)

In [ ]:
# creo el dataframe para la sumision:
dataKaggle = pd.DataFrame(ids ,columns={'id'})

In [ ]:
dataKaggle['target'] = prediction

In [ ]:
dataKaggle.head()

In [ ]:
dataKaggle.to_csv("predictionXGBoost.csv", index = False)

In [ ]:
# 566214.13293

In [ ]:
#mejor modelo xgboost:
# con los datos del gridsearch defino el mejor modelo de xgboost que encontre:
#mejor_xgb =  xgboost.XGBRegressor(
#     learning_rate =0.1,
#     n_estimators=1000,
#     max_depth=7,
#     min_child_weight=2,
#     gamma=0.1,
#     subsample=0.95,
#     colsample_bytree=0.8,
#     objective= 'reg:squarederror',
#     nthread=4,
#     scale_pos_weight=1)

In [ ]:
#mejor modelo xgboost:
# con los datos del gridsearch defino el mejor modelo de xgboost que encontre:
#mejor_xgb =  xgboost.XGBRegressor(
#     learning_rate =0.1,
#     n_estimators=1000,
#     max_depth=7,
#     min_child_weight=3,
#     gamma=0.5,
#     subsample=0.95,
#     colsample_bytree=0.8,
#     objective= 'reg:squarederror',
#     nthread=4,
#     scale_pos_weight=1)[mejor2]